NAME: **SULEGAMA JHANSI**

COURSE: **DATA SCIENCE (3PM TO 5PM)**

INCHARGE: **PANAM SRAVANI**


#16KNN

In [ ]:
# Step 1: Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import lightgbm as lgb
import xgboost as xgb

# Step 2: Load the Titanic dataset



In [238]:
zoo=pd.read_csv('/content/Zoo.csv')
zoo.head(30)

,animal name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,type
0,aardvark,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
1,antelope,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
2,bass,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
3,bear,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
4,boar,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1
5,buffalo,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
6,calf,1,0,0,1,0,0,0,1,1,1,0,0,4,1,1,1,1
7,carp,0,0,1,0,0,1,0,1,1,0,0,1,0,1,1,0,4
8,catfish,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
9,cavy,1,0,0,1,0,0,0,1,1,1,0,0,4,0,1,0,1


In [239]:
zoo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   animal name  101 non-null    object
 1   hair         101 non-null    int64 
 2   feathers     101 non-null    int64 
 3   eggs         101 non-null    int64 
 4   milk         101 non-null    int64 
 5   airborne     101 non-null    int64 
 6   aquatic      101 non-null    int64 
 7   predator     101 non-null    int64 
 8   toothed      101 non-null    int64 
 9   backbone     101 non-null    int64 
 10  breathes     101 non-null    int64 
 11  venomous     101 non-null    int64 
 12  fins         101 non-null    int64 
 13  legs         101 non-null    int64 
 14  tail         101 non-null    int64 
 15  domestic     101 non-null    int64 
 16  catsize      101 non-null    int64 
 17  type         101 non-null    int64 
dtypes: int64(17), object(1)
memory usage: 14.3+ KB


In [240]:
#  Data Preprocessing (Updated Version)
print("\nPreprocessing the data...")

# Check for missing values
print("\nMissing values before handling:")
print(df.isnull().sum())

# Handle missing values
df = df.fillna(df.mean(numeric_only=True))  # For numerical columns
df = df.fillna(df.mode().iloc[0])  # For categorical columns


Preprocessing the data...

Missing values before handling:
sepal length (cm)    0
sepal width (cm)     0
petal length (cm)    0
petal width (cm)     0
target               0
dtype: int64


In [243]:
# Separate features and target
X = zoo.drop('animal name', axis=1)  # Changed 'animal_type' to 'animal name'
y = zoo['animal name']  # Changed 'animal_type' to 'animal name'

In [244]:
# Identify and handle non-numeric columns more thoroughly
non_numeric_cols = X.select_dtypes(include=['object', 'bool']).columns
print("\nNon-numeric columns:", non_numeric_cols)

for col in X.columns:
    # Check if column contains boolean strings
    unique_vals = set(str(x).lower() for x in X[col].unique())
    if {'true', 'false'}.issubset(unique_vals):
        X[col] = X[col].replace({'true': 1, 'false': 0, 'True': 1, 'False': 0}, inplace=False)
        X[col] = pd.to_numeric(X[col], errors='coerce')
    # Convert actual boolean columns
    elif X[col].dtype == 'bool':
        X[col] = X[col].astype(int)
    # Convert other categorical columns with few unique values
    elif X[col].dtype == 'object' and X[col].nunique() < 10:
        X[col] = LabelEncoder().fit_transform(X[col])
    # Drop columns with many unique values (like names)
    elif X[col].dtype == 'object' and X[col].nunique() >= 10:
        X = X.drop(col, axis=1)


Non-numeric columns: Index([], dtype='object')


In [245]:
# Ensure all remaining columns are numeric
X = X.apply(pd.to_numeric, errors='coerce')
X = X.dropna(axis=1)  # Drop columns that couldn't be converted

In [246]:
print("\nColumns after conversion:")
print(X.dtypes)



Columns after conversion:
hair        int64
feathers    int64
eggs        int64
milk        int64
airborne    int64
aquatic     int64
predator    int64
toothed     int64
backbone    int64
breathes    int64
venomous    int64
fins        int64
legs        int64
tail        int64
domestic    int64
catsize     int64
type        int64
dtype: object


In [247]:
# Handle outliers using Z-score
z_scores = stats.zscore(X)
abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3).all(axis=1)
X = X[filtered_entries]
y = y[filtered_entries]

print("\nNumber of rows after removing outliers:", len(X))



Number of rows after removing outliers: 93


In [248]:
# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Encode target variable
le = LabelEncoder()
y = le.fit_transform(y)